# XChecker toy examples: AXp and CXp validation

This notebook mirrors `example_validation.py` and adds CXp validation plus
benchmark selection (e.g., `29_Pima` or `xd6`).

In [11]:
from pathlib import Path
import sys
import os


def find_repo_root(start=None):
    start = Path(start or Path.cwd()).resolve()
    for p in [start] + list(start.parents):
        if (p / "XChecker").exists() and (p / "RFxpl").exists():
            return p
    raise FileNotFoundError("Could not locate repo root containing XChecker and RFxpl")


REPO_ROOT = find_repo_root()
PROJECT_ROOT = REPO_ROOT

# Ensure imports work from the repo root
sys.path.insert(0, str(REPO_ROOT))
sys.path.insert(0, str(REPO_ROOT / "RFxpl"))

# XReason-RFs (optional)
XREASON_SRC = REPO_ROOT / "XReason-RFs" / "src"
if XREASON_SRC.exists():
    sys.path.insert(0, str(XREASON_SRC))

print("Repo root:", REPO_ROOT)

Repo root: /Users/yizza/Documents/Projects/chkxp/tools


In [12]:
# Choose a benchmark
# - 29_Pima is available under XChecker/experiments/tests/29_Pima
# - xd6 requires you to provide a CSV dataset path (not present in this repo)

BENCHMARK = "29_Pima"  # or "xd6"

BENCHMARKS = {
    "29_Pima": {
        "csv": PROJECT_ROOT / "XChecker/experiments/tests/29_Pima/29_Pima.csv",
        "pickle": PROJECT_ROOT / "XChecker/experiments/tests/29_Pima/29_Pima_nbestim_5_maxdepth_4.mod.pkl",
        "encoding": PROJECT_ROOT / "XChecker/experiments/tests/29_Pima/29_Pima_nbestim_5_maxdepth_4.smt",
        "pyxai_log_abd": None,
        "pyxai_log_con": None,
    },
    "xd6": {
        "csv": PROJECT_ROOT / "XChecker/experiments/tests/xd6/xd6.csv",
        #"pickle": PROJECT_ROOT / "RFxpl/Classifiers/xd6/xd6_nbestim_4_maxdepth_4.mod.pkl",
        "pickle": PROJECT_ROOT / "XChecker/experiments/xd6/xd6_nbestim_4_maxdepth_4.mod.pkl",
        "encoding": None,  # set to your SMT encoding if you want XReason checks
        "pyxai_log_abd": None,
        "pyxai_log_con": None,
    },
}

cfg = BENCHMARKS[BENCHMARK]
print("Benchmark:", BENCHMARK)
print(cfg)

Benchmark: 29_Pima
{'csv': PosixPath('/Users/yizza/Documents/Projects/chkxp/tools/XChecker/experiments/tests/29_Pima/29_Pima.csv'), 'pickle': PosixPath('/Users/yizza/Documents/Projects/chkxp/tools/XChecker/experiments/tests/29_Pima/29_Pima_nbestim_5_maxdepth_4.mod.pkl'), 'encoding': PosixPath('/Users/yizza/Documents/Projects/chkxp/tools/XChecker/experiments/tests/29_Pima/29_Pima_nbestim_5_maxdepth_4.smt'), 'pyxai_log_abd': None, 'pyxai_log_con': None}


In [13]:
import numpy as np
import re

from XChecker.checker import Validator, ValidationResult
from XChecker.checker import NumericDomain
from XChecker.adapters import RFxplExplainerR, XReasonExplainerS
from XChecker.adapters.rfxpl_R import RF_Model

from xrf import RFSklearn, Dataset, Forest
from options import Options as RFOptions

In [14]:
# Load model + dataset and build feature domains


def load_benchmark(cfg):
    csv_path = cfg.get("csv")
    pickle_path = cfg.get("pickle")
    
    if not pickle_path or not Path(pickle_path).exists():
        raise FileNotFoundError(f"Missing model pickle: {pickle_path}")    
    if not csv_path or not Path(csv_path).exists():
        raise FileNotFoundError(
            f"Missing CSV dataset for benchmark. Set cfg['csv'] to a valid path.Got: {csv_path}")

    model_path = Path(pickle_path).name
    match = re.search(r"nbestim_(\d+)_maxdepth_(\d+)", model_path)
    n_trees, maxdepth = int(match.group(1)), int(match.group(2))

    opts = RFOptions(None)
    opts.files = [str(pickle_path), str(csv_path)]

    rf_md = RFSklearn(from_file=opts.files[0])
    data = Dataset(filename=opts.files[1], use_categorical=False)

    feature_names, target_names = data.features, data.targets
    forest = Forest(rf_md.estimators(), data.m_features_)

    # Feature domains for witness generation
    min_max_dom = [
        NumericDomain(data.X[:, i].min(), data.X[:, i].max())
        for i in range(data.X.shape[1])
    ]

    ml_model = RF_Model(forest)
    return rf_md, data, feature_names, target_names, n_trees, maxdepth, min_max_dom, ml_model


rf_md, data, feature_names, target_names, n_trees, maxdepth, domains, ml_model = load_benchmark(cfg)
        
print("Loaded", n_trees, " trees, maxdepth: ", maxdepth)

c nof features: 8
c nof classes: 2
c nof samples: 768
min: 21 | max: 31
Loaded 5  trees, maxdepth:  4


In [15]:
# Initialize explainers

explainer_r = RFxplExplainerR(
    rf_md, feature_names, target_names,
    domains=domains,
    verbose=False,
)

explainer_s = None
encoding_path = cfg.get("encoding")
if encoding_path and Path(encoding_path).exists():
    try:
        explainer_s = XReasonExplainerS(
            ml_model,
            rf_md.feature_names,
            target_names,
            encoding=str(encoding_path),
        )
        print("XReason explainer S enabled")
    except Exception as exc:
        print("XReason explainer S unavailable:", exc)
else:
    print("XReason explainer S disabled (no encoding provided)")

validator = Validator(
    explainer_t=None,
    explainer_r=explainer_r,
    explainer_s=explainer_s,
    ml_model=ml_model,
    verbose=True,
)

min: 21 | max: 31
XReason explainer S enabled


In [16]:
# Pick an instance to validate
# Use the first row by default

instance = np.array(data.X[0])
prediction = ml_model.predict(instance)

print("Instance:", instance)
print("Prediction:", prediction)

Instance: [  6.    148.     72.     35.      0.     33.6     0.627  50.   ]
Prediction: 1


In [17]:
# Validate AXp

# Option A: use RFxpl to produce an AXp
expl_axp = explainer_r.findaxp((instance, prediction))

# Option B: set your own explanation
# expl_axp = {0, 1, 2}

print("AXp explanation:", sorted(expl_axp))

report_axp = validator.validate_axp((instance, prediction), set(expl_axp))

print("AXp Validation Report")
print(report_axp)

if report_axp.result == ValidationResult.CORRECT:
    print("AXp: CORRECT and MINIMAL")
elif report_axp.result == ValidationResult.NON_MINIMAL:
    print("AXp: CORRECT but NON_MINIMAL")
    print("Real explanation:", sorted(report_axp.real_explanation))
elif report_axp.result == ValidationResult.INCORRECT:
    print("AXp: INCORRECT")
    if report_axp.witness is not None:
        print("Witness:", report_axp.witness)
else:
    print("AXp: ERROR")
    print("Errors:", report_axp.errors)

AXp explanation: [1, 4, 5, 6, 7]
AXp Validation Report
Result: non_minimal
Valid: False
Minimal: False
Real explanation: [1, 6, 7]
Witness checks: 3
Proof checks: 3
AXp: CORRECT but NON_MINIMAL
Real explanation: [1, 6, 7]


In [20]:
# Validate CXp

# Option A: use RFxpl to produce a CXp
expl_cxp = explainer_r.findcxp((instance, prediction))

# Option B: set your own explanation
expl_cxp = {0, 1, 2}

print("CXp explanation:", sorted(expl_cxp))

report_cxp = validator.validate_cxp((instance, prediction), set(expl_cxp))

print("CXp Validation Report")
print(report_cxp)

if report_cxp.result == ValidationResult.CORRECT:
    print("CXp: CORRECT and MINIMAL")
elif report_cxp.result == ValidationResult.NON_MINIMAL:
    print("CXp: CORRECT but NON_MINIMAL")
    print("Real explanation:", sorted(report_cxp.real_explanation))
elif report_cxp.result == ValidationResult.INCORRECT:
    print("CXp: INCORRECT")
    if report_cxp.witness is not None:
        print("Witness:", report_cxp.witness)
else:
    print("CXp: ERROR")
    print("Errors:", report_cxp.errors)

AttributeError: 'SATEncoder' object has no attribute 'soft'

In [ ]:
# Optional: parse PyXAI logs like RFxpl/check_axp_toy.py or check_cxp_toy.py
# Provide paths in cfg['pyxai_log_abd'] and cfg['pyxai_log_con']


def parse_pyxai_axp_log(log_path):
    i_list = []
    i_pred_list = []
    expl_list = []
    with open(log_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("i:"):
                values = list(map(float, line[2:].strip().split(',')))
                i_list.append(values)
            elif line.startswith("pred:"):
                values = int(line[5:].strip())
                i_pred_list.append(values)
            elif line.startswith("expl:"):
                raw = line[5:].strip().strip('[]')
                values = list(map(int, raw.split(','))) if raw else []
                expl_list.append(values)
    return i_list, i_pred_list, expl_list


axp_log = cfg.get("pyxai_log_abd")
if axp_log and Path(axp_log).exists():
    i_list, i_pred_list, expl_list = parse_pyxai_axp_log(axp_log)
    print("Loaded", len(i_list), "AXp instances from log")
else:
    print("No AXp log provided")

In [ ]:
# Optional: validate a log entry (AXp)
# Make sure you loaded i_list/i_pred_list/expl_list from the log above.

# idx = 0
# instance = np.array(i_list[idx])
# prediction = i_pred_list[idx]
# expl = set([i - 1 for i in expl_list[idx]])  # RFxpl expects 0-indexed
# report = validator.validate_axp((instance, prediction), expl)
# print(report)

In [ ]:
# Optional: parse PyXAI CXp logs like RFxpl/check_cxp_toy.py


def parse_pyxai_cxp_log(log_path):
    i_list = []
    i_pred_list = []
    expl_list = []
    pyxai_wit_list = []
    pyxai_pred_wit_list = []
    with open(log_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("i:"):
                values = list(map(float, line[2:].strip().split(',')))
                i_list.append(values)
            elif line.startswith("pred:"):
                values = int(line[5:].strip())
                i_pred_list.append(values)
            elif line.startswith("expl:"):
                raw = line[5:].strip().strip('[]')
                values = list(map(int, raw.split(','))) if raw else []
                expl_list.append(values)
            elif line.startswith("wit:"):
                values = list(map(float, line[4:].strip().split(',')))
                pyxai_wit_list.append(values)
            elif line.startswith("pred of wit:"):
                values = int(line[12:].strip())
                pyxai_pred_wit_list.append(values)
    return i_list, i_pred_list, expl_list, pyxai_wit_list, pyxai_pred_wit_list


cxp_log = cfg.get("pyxai_log_con")
if cxp_log and Path(cxp_log).exists():
    i_list, i_pred_list, expl_list, pyxai_wit_list, pyxai_pred_wit_list = parse_pyxai_cxp_log(cxp_log)
    print("Loaded", len(i_list), "CXp instances from log")
else:
    print("No CXp log provided")

In [ ]:
# Optional: validate a log entry (CXp)
# Make sure you loaded i_list/i_pred_list/expl_list from the log above.

# idx = 0
# instance = np.array(i_list[idx])
# prediction = i_pred_list[idx]
# expl = set([i - 1 for i in expl_list[idx]])  # RFxpl expects 0-indexed
# report = validator.validate_cxp((instance, prediction), expl)
# print(report)